<a href="https://colab.research.google.com/github/Abdul-RehmaN47/Artificial-Intelligence/blob/main/GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install gradio openai gtts pydub numpy requests groq openai-whisper  # Install necessary Python libraries for the project
!apt-get install -y ffmpeg  # Install FFmpeg for audio processing

import os  # Provides functions to interact with the operating system
import gradio as gr  # Gradio is used to create a web-based user interface
import whisper  # Whisper is a speech recognition library by OpenAI for transcribing audio
from gtts import gTTS  # gTTS (Google Text-to-Speech) is used to convert text responses into speech
from IPython.display import Audio  # Allows the display of audio files in Jupyter notebooks
from groq import Groq  # Groq is used to interact with the Groq API for language model responses
import openai  # OpenAI library for interacting with OpenAI's API, though not used directly here
import numpy as np  # NumPy is used for numerical operations (not used in this script)

import os  # Operating system interactions (repeated import, unnecessary)
import gradio as gr  # Web-based interface for the voice chat application (repeated import)
import whisper  # Speech recognition for converting voice to text (repeated import)
from gtts import gTTS  # Converts text to speech (repeated import)
from groq import Groq, GroqError  # Groq API client and error handling for generating text responses
from typing import Tuple, Union  # Used to specify complex function return types

# Initialize Whisper model for audio transcription
model = whisper.load_model("base")  # Loads the base Whisper model for speech-to-text conversion

# Initialize Groq API client with the API key directly
api_key = "groq_api_key"  # API key for authenticating with the Groq service
try:
    client = Groq(api_key=api_key)  # Create a Groq client for interacting with the API
except Exception as e:
    raise RuntimeError(f"Failed to initialize Groq client: {e}")  # Handle errors in initializing the Groq client

def transcribe_and_respond(audio: str) -> Tuple[str, Union[str, None]]:
    try:
        # Step 1: Transcribe the audio using Whisper
        transcription = model.transcribe(audio)  # Convert the input audio to text
        user_input = transcription['text']  # Extract the transcribed text from the Whisper output

        # Step 2: Generate a response using Groq API and LLaMA model
        try:
            chat_completion = client.chat.completions.create(
                messages=[{"role": "user", "content": user_input}],
                model="llama3-8b-8192",  # Use the LLaMA model for generating responses
            )
            response_text = chat_completion.choices[0].message.content  # Extract the response text
        except GroqError as e:
            return f"Error in Groq API call: {e}", None  # Handle Groq API errors

        # Step 3: Convert the response text to speech using gTTS
        tts = gTTS(response_text)  # Convert the response text to speech using gTTS
        audio_path = "response.mp3"  # Set the output file name for the speech audio
        tts.save(audio_path)  # Save the generated speech to a file

        return response_text, audio_path  # Return the response text and path to the speech file

    except FileNotFoundError:
        return "Error: Audio file not found.", None  # Handle missing audio file errors
    except whisper.WhisperError as e:
        return f"Error in transcription: {e}", None  # Handle transcription errors specific to Whisper
    except Exception as e:
        return f"An unexpected error occurred: {e}", None  # Catch all other unexpected errors

# Gradio interface for real-time interaction
interface = gr.Interface(
    fn=transcribe_and_respond,  # Use the transcribe_and_respond function as the main processing function
    inputs=gr.Audio(type="filepath"),  # Accept audio file input from the user
    outputs=[gr.Textbox(label="Response"), gr.Audio(label="Voice Response")],  # Display text and voice response
    live=True  # Enable real-time processing
)

interface.launch()  # Launch the Gradio web interface


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a9da9386596874c97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
